## Загрузим нужные библиотеки

Идеи:
~~1) Переменная - количество авторов. ~~

2) OneHotEncoder - авторов в число (автор чувствует статью)

~~3) Отдельно выделить без авторов?~~

4) Определить популярность слов в теге, используя гугл трендс за период --> Интересно, но сложно --> **Парсинг**

5) Определять выходные (например). => Поиграться с датой (мб часы утренние/вечерние)

***6) Заменить категорию, сессию, автора (и/или) на среднее значение по frp (например) (Если много, то взять из этого максимум)*** 

7) Аналогично с тегами (Узнать че за теги (изи) + популярность) -!!! (Траблы с тегами видимости) !!! --> **Парсинг**

8) Распарсить сайт для поиска авторов и посчитать их среднюю популярность тип. (За 10 статей, например) -!!! (Траблы с тегами видимости) !!! --> **Парсинг**

9) Получить длину слов в статье (Влияет как минимум на процент читателей полность прочитавших статью) --> **Парсинг**

~~10) Предсказать на основе View и Depth frp~~

11) Сгруппировать по сессии??

12) Добавить флаг на наличие в статье картинки --> **Парсинг**

13) Курс доллара в этот момент?? --> **Парсинг**

14) Ансабль разных моделей --> ? хз, XGB это по идее уже и есть ансабль. Только ансамбль лесов; Попробовать **другие модели??**

15) Масштабирование данных --> По идее для деревьев нет смысла

16) Timestamp

17) Для авторов через парсинг найти их view

In [1]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV


In [2]:
import numpy as np
import xgboost as xg
from sklearn.model_selection import cross_validate

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
def data_encoding(df_tr, df_tst):
    # Создаем объединенный датафрейм трейна и теста. 
    # Заменим там перемнные и разобьем его обратно
    
    frames = [df_tr[["category", "session", "authors"]], df_tst[["category", "session", "authors"]]]
    df_temp = pd.concat(frames)
    
    # Заменяем категориальные значения на int
    
    df_temp["category"] = df_temp["category"].astype('category')
    df_temp["category"] = df_temp["category"].cat.codes
    df_temp["category"] = df_temp["category"].astype('int')
    
    df_temp["authors"] = df_temp["authors"].astype('category')
    df_temp["authors"] = df_temp["authors"].cat.codes
    df_temp["authors"] = df_temp["authors"].astype('int')
    
    df_temp["session"] = df_temp["session"].astype('category')
    df_temp["session"] = df_temp["session"].cat.codes
    df_temp["session"] = df_temp["session"].astype('int')
    
    #df_temp["tags"] = df_temp["tags"].astype('category')
    #df_temp["tags"] = df_temp["tags"].cat.codes
    #df_temp["tags"] = df_temp["tags"].astype('int')
    
    # Мерджим полученные данные с траэйном и тестом
    
    df_tr = pd.merge(
        df_tr,
        df_temp,
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_tst = pd.merge(
        df_tst,
        df_temp,
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )

    # Добавляем обработку категорий в медианыые значения по своей группе
    
    
    
    
    return df_tr, df_tst

In [56]:
def category_encoding(df_tr, df_tst):
    
    frames = [df_tr.copy(), df_tst.copy()]
    df_temp = pd.concat(frames)
    
    df_temp = pd.merge(
        df_temp,
        df_temp.groupby('category').median()[['views', 'depth', 'full_reads_percent']],
        how="left",
        left_on = 'category',
        #left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_median_cat"),
        copy=True,
        indicator=False, 
        validate=None,
    )
    
    # Мерджим полученные данные с траэйном и тестом
    
    df_tr = pd.merge(
        df_tr,
        df_temp[['views_median_cat', 'depth_median_cat', 'full_reads_percent_median_cat']],
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )

    df_tst = pd.merge(
        df_tst,
        df_temp[['views_median_cat', 'depth_median_cat', 'full_reads_percent_median_cat']],
        how="left",
        left_index=True,
        right_index=True,
        sort=False,
        suffixes=("", "_code"),
        copy=True,
        indicator=False,
        validate=None,
    )

    return df_tr, df_tst

In [7]:
def feature_engineering(df_tr, df_tst):
    
    # Добавим число тегов и авторов
    
    for dataset in [df_tr, df_tst]:
        
        dataset['n_authors'] = dataset['authors'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )
        dataset['n_tags'] = dataset['tags'].map(lambda x: 0 if x.split(',') == ['[]'] else len(x.split(',')) )
        
        # Добавим день, меся, год
        
        dataset['day'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%d").astype(int)
        dataset['mounth'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%m").astype(int)
        dataset['year'] = pd.to_datetime(dataset['publish_date']).dt.strftime("%y").astype(int)
        
    return df_tr, df_tst

Выполним загрузу датсета

In [268]:
df_train = pd.read_csv("content/train_dataset_train.csv", index_col= 0)
df_test = pd.read_csv("content/test_dataset_test.csv", index_col= 0)
sample = pd.read_csv("content/sample_solution.csv", index_col= 0)

#### Кодируем признаки

In [269]:
df_train, df_test = data_encoding(df_train, df_test)

#### Добавляем день месяц год, число авторов и тегов

In [270]:
df_train, df_test = feature_engineering(df_train, df_test)

#### Добавляем для каждой категории среднее значение целевого признака

In [271]:
df_train, df_test = category_encoding(df_train, df_test)

In [272]:
df_train.columns

Index(['title', 'publish_date', 'session', 'authors', 'ctr', 'category',
       'tags', 'views', 'depth', 'full_reads_percent', 'category_code',
       'session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
       'year', 'views_median_cat', 'depth_median_cat',
       'full_reads_percent_median_cat'],
      dtype='object')

#### Обработка авторов и тегов

Создаем колонки

In [273]:
df_train['views_mean_tags'] = -1.0
df_train['depth_mean_tags'] = -1.0
df_train['frp_mean_tags'] = -1.0

Формируем словари для тегов

In [274]:
%%time
tags_count_d = {'[]': 0}
tag_sum_v_d = {'[]': 0}
tag_sum_d_d = {'[]': 0}
tag_sum_frp_d = {'[]': 0}

# По каждой строке
for i in range(df_train.shape[0]):
    
    # Если нет тега, то увеличиваем счетчик пустых тэгов и записываем сумму
    if df_train.tags[i] == "[]":
        tags_count_d['[]'] += 1
        tag_sum_v_d['[]'] += df_train.views[i]
        tag_sum_d_d['[]'] += df_train.depth[i]
        tag_sum_frp_d['[]'] += df_train.full_reads_percent[i]
        continue
    
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_train.tags[i].split("['")[1].split("']")[0].split("', '"):
        
        # Если такой тег уже существует, суммируем значения
        if x in list(tags_count_d.keys()):
            tags_count_d[x] += 1
            tag_sum_v_d[x] += df_train.views[i]
            tag_sum_d_d[x] += df_train.depth[i]
            tag_sum_frp_d[x] += df_train.full_reads_percent[i]
            
        else: 
            tags_count_d[x] = 1
            tag_sum_v_d[x] = df_train.views[i]
            tag_sum_d_d[x] = df_train.depth[i]
            tag_sum_frp_d[x] = df_train.full_reads_percent[i]


Wall time: 8.37 s


Записываем в датафрейм максимальное среднее значение по тегам

In [275]:
# Для пустых сразу считаем среднее
tag_sum_v_d['[]'] = tag_sum_v_d['[]'] / tags_count_d['[]']
tag_sum_d_d['[]'] = tag_sum_d_d['[]'] / tags_count_d['[]']
tag_sum_frp_d['[]'] = tag_sum_frp_d['[]'] / tags_count_d['[]']

# По каждой строке
for i in range(df_train.shape[0]):
    
    # Если нет тега
    if df_train.tags[i] == "[]":
        df_train['views_mean_tags'][i] = tag_sum_v_d['[]']
        df_train['depth_mean_tags'][i] = tag_sum_d_d['[]']
        df_train['frp_mean_tags'][i] = tag_sum_frp_d['[]']
        continue
    
    maximum_v = 0
    maximum_d = 0
    maximum_frp = 0
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_train.tags[i].split("['")[1].split("']")[0].split("', '"):
        # Считаем среднее значение для данного тега
        mean_v = tag_sum_v_d[x] / tags_count_d[x]
        mean_d = tag_sum_d_d[x] / tags_count_d[x]
        mean_frp = tag_sum_frp_d[x] / tags_count_d[x]
        
        # Запоминаем максимальное среднее значение
        if mean_v > maximum_v:
            maximum_v = mean_v
        if mean_d > maximum_d:
            maximum_d = mean_d
        if mean_frp > maximum_frp:
            maximum_frp = mean_frp
        
    df_train['views_mean_tags'][i] = maximum_v
    df_train['depth_mean_tags'][i] = maximum_d
    df_train['frp_mean_tags'][i] = maximum_frp

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to b

Аналогично добавляем колонки в тестовом датасете

In [276]:
df_test['views_mean_tags'] = -1.0
df_test['depth_mean_tags'] = -1.0
df_test['frp_mean_tags'] = -1.0

# Для пустых сразу считаем среднее
#tag_sum_v_d['[]'] = tag_sum_v_d['[]'] / tags_count_d['[]']
#tag_sum_d_d['[]'] = tag_sum_d_d['[]'] / tags_count_d['[]']
#tag_sum_frp_d['[]'] = tag_sum_frp_d['[]'] / tags_count_d['[]']

# По каждой строке
for i in range(df_test.shape[0]):
    
    # Если нет тега
    if df_test.tags[i] == "[]":
        df_test['views_mean_tags'][i] = tag_sum_v_d['[]']
        df_test['depth_mean_tags'][i] = tag_sum_d_d['[]']
        df_test['frp_mean_tags'][i] = tag_sum_frp_d['[]']
        continue
    
    maximum_v = 0
    maximum_d = 0
    maximum_frp = 0
    
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_test.tags[i].split("['")[1].split("']")[0].split("', '"):
        
        # Если встретили незнакомый тег:
        if x not in tags_count_d.keys():
            mean_v = 0
            mean_d = 0
            mean_frp = 0
            continue
        
        # Если тег встечался нами ранее
        # Считаем среднее значение для данного тега
        mean_v = tag_sum_v_d[x] / tags_count_d[x]
        mean_d = tag_sum_d_d[x] / tags_count_d[x]
        mean_frp = tag_sum_frp_d[x] / tags_count_d[x]
        
        # Запоминаем максимальное среднее значение
        if mean_v > maximum_v:
            maximum_v = mean_v
        if mean_d > maximum_d:
            maximum_d = mean_d
        if mean_frp > maximum_frp:
            maximum_frp = mean_frp
        
    df_test['views_mean_tags'][i] = maximum_v
    df_test['depth_mean_tags'][i] = maximum_d
    df_test['frp_mean_tags'][i] = maximum_frp

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to b

In [277]:
# 10 тегов совершенно новые; Надо бы заменпить их на медианное значение
len(df_test[df_test['views_mean_tags'] == 0])

10

Заменяем незнакомые теги на медианные

In [278]:
df_test['views_mean_tags'][df_test['views_mean_tags'] == 0] = df_test['views_mean_tags'].median()
df_test['depth_mean_tags'][df_test['depth_mean_tags'] == 0] = df_test['depth_mean_tags'].median()
df_test['frp_mean_tags'][df_test['frp_mean_tags'] == 0] = df_test['frp_mean_tags'].median()

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [279]:
len(df_test[df_test['views_mean_tags'] == 0])

0

Формируем словари для авторов

Создаем колонки

In [280]:
df_train['views_mean_authors'] = -1.0
df_train['depth_mean_authors'] = -1.0
df_train['frp_mean_authors'] = -1.0

In [281]:
%%time

authors_count_d = {'[]': 0}
authors_sum_v_d = {'[]': 0}
authors_sum_d_d = {'[]': 0}
authors_sum_frp_d = {'[]': 0}

# По каждой строке
for i in range(df_train.shape[0]):
    
    # Если нет тега, то увеличиваем счетчик пустых тэгов и записываем сумму
    if df_train.authors[i] == "[]":
        authors_count_d['[]'] += 1
        authors_sum_v_d['[]'] += df_train.views[i]
        authors_sum_d_d['[]'] += df_train.depth[i]
        authors_sum_frp_d['[]'] += df_train.full_reads_percent[i]
        continue
    
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_train.authors[i].split("['")[1].split("']")[0].split("','"):
        
        # Если такой тег уже существует, суммируем значения
        if x in list(authors_count_d.keys()):
            authors_count_d[x] += 1
            authors_sum_v_d[x] += df_train.views[i]
            authors_sum_d_d[x] += df_train.depth[i]
            authors_sum_frp_d[x] += df_train.full_reads_percent[i]
            
        else: 
            authors_count_d[x] = 1
            authors_sum_v_d[x] = df_train.views[i]
            authors_sum_d_d[x] = df_train.depth[i]
            authors_sum_frp_d[x] = df_train.full_reads_percent[i]

Wall time: 1.77 s


In [282]:
# Для пустых сразу считаем среднее
authors_sum_v_d['[]'] = authors_sum_v_d['[]'] / authors_count_d['[]']
authors_sum_d_d['[]'] = authors_sum_d_d['[]'] / authors_count_d['[]']
authors_sum_frp_d['[]'] = authors_sum_frp_d['[]'] / authors_count_d['[]']

# По каждой строке
for i in range(df_train.shape[0]):
    
    # Если нет тега
    if df_train.authors[i] == "[]":
        df_train['views_mean_authors'][i] = authors_sum_v_d['[]']
        df_train['depth_mean_authors'][i] = authors_sum_d_d['[]']
        df_train['frp_mean_authors'][i] = authors_sum_frp_d['[]']
        continue
    
    maximum_v = 0
    maximum_d = 0
    maximum_frp = 0
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_train.authors[i].split("['")[1].split("']")[0].split("','"):
        # Считаем среднее значение для данного тега
        mean_v = authors_sum_v_d[x] / authors_count_d[x]
        mean_d = authors_sum_d_d[x] / authors_count_d[x]
        mean_frp = authors_sum_frp_d[x] / authors_count_d[x]
        
        # Запоминаем максимальное среднее значение
        if mean_v > maximum_v:
            maximum_v = mean_v
        if mean_d > maximum_d:
            maximum_d = mean_d
        if mean_frp > maximum_frp:
            maximum_frp = mean_frp
        
    df_train['views_mean_authors'][i] = maximum_v
    df_train['depth_mean_authors'][i] = maximum_d
    df_train['frp_mean_authors'][i] = maximum_frp

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda\

Аналогично для теста:

In [286]:
df_test['views_mean_authors'] = -1.0
df_test['depth_mean_authors'] = -1.0
df_test['frp_mean_authors'] = -1.0

## Для пустых сразу считаем среднее
#authors_sum_v_d['[]'] = authors_sum_v_d['[]'] / authors_count_d['[]']
#authors_sum_d_d['[]'] = authors_sum_d_d['[]'] / authors_count_d['[]']
#authors_sum_frp_d['[]'] = authors_sum_frp_d['[]'] / authors_count_d['[]']

# По каждой строке
for i in range(df_test.shape[0]):
    
    # Если нет тега
    if df_test.authors[i] == "[]":
        df_test['views_mean_authors'][i] = authors_sum_v_d['[]']
        df_test['depth_mean_authors'][i] = authors_sum_d_d['[]']
        df_test['frp_mean_authors'][i] = authors_sum_frp_d['[]']
        continue
    
    maximum_v = 0
    maximum_d = 0
    maximum_frp = 0
    # Если тег есть, парсим его для выделения занчений тегов    
    for x in df_test.authors[i].split("['")[1].split("']")[0].split("','"):
        
        # Если встретили незнакомый тег:
        if x not in tags_count_d.keys():
            mean_v = 0
            mean_d = 0
            mean_frp = 0
            continue
        
        # Считаем среднее значение для данного тега
        mean_v = authors_sum_v_d[x] / authors_count_d[x]
        mean_d = authors_sum_d_d[x] / authors_count_d[x]
        mean_frp = authors_sum_frp_d[x] / authors_count_d[x]
        
        # Запоминаем максимальное среднее значение
        if mean_v > maximum_v:
            maximum_v = mean_v
        if mean_d > maximum_d:
            maximum_d = mean_d
        if mean_frp > maximum_frp:
            maximum_frp = mean_frp
        
    df_test['views_mean_authors'][i] = maximum_v
    df_test['depth_mean_authors'][i] = maximum_d
    df_test['frp_mean_authors'][i] = maximum_frp

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\Anaconda\lib\site-packages\ipyk

In [287]:
len(df_test[df_test['views_mean_authors'] == 0])

1771

Большая часть авторов - новая

In [295]:
df_test['views_mean_authors'][df_test['views_mean_authors'] == 0] = df_test['views_mean_authors'][df_test['views_mean_authors'] != 0].median()
df_test['depth_mean_authors'][df_test['depth_mean_authors'] == 0] = df_test['depth_mean_authors'][df_test['views_mean_authors'] != 0].median()
df_test['frp_mean_authors'][df_test['frp_mean_authors'] == 0] = df_test['frp_mean_authors'][df_test['views_mean_authors'] != 0].median()

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [296]:
len(df_test[df_test['views_mean_authors'] == 0])

0

Выделяем X и y

In [298]:
X = df_train.drop(['title', 'publish_date', 'session', 'authors', 'category', 'tags',
                  'views', 'depth', 'full_reads_percent'], axis = 1)

y = df_train[['views', 'depth', 'full_reads_percent']]

In [34]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [299]:
model_views = xg.XGBRegressor()
model_depth = xg.XGBRegressor()
model_full_reads_percent = xg.XGBRegressor()

Проверка нужных фичей для первой модели

In [314]:
%%time

X_tmp = X.copy()
X_tmp = X_tmp.drop(['views_mean_authors','depth_mean_authors', 'frp_mean_authors'], axis = 1)

scores_v = cross_validate(model_views, X_tmp, y['views'], cv=5, scoring='r2')
just_score = scores_v['test_score'].mean()
print('Используя все фичи: ', just_score)

max_score = just_score

while True:
    
    worsed_fiature = 'None'
    
    for fiature in X_tmp.columns:
        model_1 = xg.XGBRegressor()
        scores_v = cross_validate(model_1, 
                                  X_tmp.drop(fiature, axis = 1),
                                  y['views'], cv=5, scoring='r2')
        print('Без ',fiature,':', scores_v['test_score'].mean())
        
        if scores_v['test_score'].mean() > max_score:
            max_score = scores_v['test_score'].mean()
            worsed_fiature = fiature
    print('Худшая фича прогона: ', worsed_fiature)
    print('Максимальный скор прогона: ', max_score)
    print()
    
    # Если улучшений нет, оставляем как есть
    if worsed_fiature == 'None':
        break
    X_tmp = X_tmp.drop(worsed_fiature, axis = 1)
print("Итоговые фичи: ", X_tmp.columns)


# Максимальный скор прогона:  0.6692638219008774
# Итоговые фичи:  Index(['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
#        'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
#        'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#        'depth_mean_authors', 'frp_mean_authors'],
#       dtype='object')

Используя все фичи:  0.5173791650355881
Без  ctr : 0.6504322704290704
Без  category_code : 0.5152585450705859
Без  session_code : 0.5155873233487548
Без  authors_code : 0.4344493493407959
Без  n_authors : 0.5056904515271912
Без  n_tags : 0.4261323804699596
Без  day : 0.4945456227922881
Без  mounth : 0.44914306994291603
Без  year : 0.5125086025659341
Без  views_median_cat : 0.5110781213024427
Без  depth_median_cat : 0.5228167596252941
Без  full_reads_percent_median_cat : 0.5125338269612407
Без  views_mean_tags : 0.5739646770876188
Без  depth_mean_tags : 0.48492458776591896
Без  frp_mean_tags : 0.5388300605723161
Худшая фича прогона:  ctr
Максимальный скор прогона:  0.6504322704290704

Без  category_code : 0.6510252256092233
Без  session_code : 0.6551139549375221
Без  authors_code : 0.6025373671526685
Без  n_authors : 0.6684893227903455
Без  n_tags : -0.16623837558864826
Без  day : 0.6613352623501593
Без  mounth : 0.5744241179833628
Без  year : 0.6506021135983927
Без  views_median_cat : 

Аналогично для Depth

In [315]:
%%time

X_tmp = X.copy()
X_tmp = X_tmp.drop(['views_mean_authors','depth_mean_authors', 'frp_mean_authors'], axis = 1)

scores_v = cross_validate(model_depth, X_tmp, y['depth'], cv=5, scoring='r2')
just_score = scores_v['test_score'].mean()
print('Используя все фичи: ', just_score)

max_score = just_score

while True:
    
    worsed_fiature = 'None'
    
    for fiature in X_tmp.columns:
        model_1 = xg.XGBRegressor()
        scores_v = cross_validate(model_1, 
                                  X_tmp.drop(fiature, axis = 1),
                                  y['depth'], cv=5, scoring='r2')
        print('Без ',fiature,':', scores_v['test_score'].mean())
        
        if scores_v['test_score'].mean() > max_score:
            max_score = scores_v['test_score'].mean()
            worsed_fiature = fiature
    print('Худшая фича прогона: ', worsed_fiature)
    print('Максимальный скор прогона: ', max_score)
    print()
    
    # Если улучшений нет, оставляем как есть
    if worsed_fiature == 'None':
        break
    X_tmp = X_tmp.drop(worsed_fiature, axis = 1)
print("Итоговые фичи: ", X_tmp.columns)

# Максимальный скор прогона:  0.8099143688505638
# 
# Итоговые фичи:  Index(['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#        'day', 'mounth', 'views_median_cat', 'depth_median_cat',
#        'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
#        'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
#        'frp_mean_authors'],
#       dtype='object')

Используя все фичи:  0.8012668732140735
Без  ctr : 0.7898202710356479
Без  category_code : 0.8021212218690479
Без  session_code : 0.8040783936393396
Без  authors_code : 0.803326678812916
Без  n_authors : 0.8027247441772134
Без  n_tags : 0.8039918626499297
Без  day : 0.681190794822328
Без  mounth : 0.5458135334619263
Без  year : 0.8031685809692858
Без  views_median_cat : 0.7992875861699869
Без  depth_median_cat : 0.8016633853832371
Без  full_reads_percent_median_cat : 0.8029295800424568
Без  views_mean_tags : 0.8009013821133365
Без  depth_mean_tags : 0.7695783377456925
Без  frp_mean_tags : 0.8041778735989527
Худшая фича прогона:  frp_mean_tags
Максимальный скор прогона:  0.8041778735989527

Без  ctr : 0.7854708020770802
Без  category_code : 0.8040227763179495
Без  session_code : 0.8006588005556348
Без  authors_code : 0.8011905103192773
Без  n_authors : 0.7998008064859887
Без  n_tags : 0.8044848461169032
Без  day : 0.6802891289182919
Без  mounth : 0.5416820981489279
Без  year : 0.8011675

И для FRP

In [316]:
%%time

X_tmp = X.copy()
X_tmp = X_tmp.drop(['views_mean_authors','depth_mean_authors', 'frp_mean_authors'], axis = 1)

scores_v = cross_validate(model_depth, X_tmp, y['full_reads_percent'], cv=5, scoring='r2')
just_score = scores_v['test_score'].mean()
print('Используя все фичи: ', just_score)

max_score = just_score

while True:
    
    worsed_fiature = 'None'
    
    for fiature in X_tmp.columns:
        model_1 = xg.XGBRegressor()
        scores_v = cross_validate(model_1, 
                                  X_tmp.drop(fiature, axis = 1),
                                  y['full_reads_percent'], cv=5, scoring='r2')
        print('Без ',fiature,':', scores_v['test_score'].mean())
        
        if scores_v['test_score'].mean() > max_score:
            max_score = scores_v['test_score'].mean()
            worsed_fiature = fiature
    print('Худшая фича прогона: ', worsed_fiature)
    print('Максимальный скор прогона: ', max_score)
    print()
    
    # Если улучшений нет, оставляем как есть
    if worsed_fiature == 'None':
        break
    X_tmp = X_tmp.drop(worsed_fiature, axis = 1)
print("Итоговые фичи: ", X_tmp.columns)

#Максимальный скор прогона:  0.4699530691376152
#
#Итоговые фичи:  Index(['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#       'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
#       'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
#       'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#       'frp_mean_authors'],
#      dtype='object')

Используя все фичи:  0.4583854555793245
Без  ctr : 0.42839651265545003
Без  category_code : 0.45418101711556014
Без  session_code : 0.45607875604037795
Без  authors_code : 0.43840354955587524
Без  n_authors : 0.4560942967572975
Без  n_tags : 0.4534257434184113
Без  day : 0.45031121804867313
Без  mounth : 0.43451766984535906
Без  year : 0.45924496504312484
Без  views_median_cat : 0.4501437410691331
Без  depth_median_cat : 0.45681538083801076
Без  full_reads_percent_median_cat : 0.45107658356086705
Без  views_mean_tags : 0.45708376567083986
Без  depth_mean_tags : 0.45794542341127287
Без  frp_mean_tags : 0.3027289321971957
Худшая фича прогона:  year
Максимальный скор прогона:  0.45924496504312484

Без  ctr : 0.42771096198363745
Без  category_code : 0.4540210592765039
Без  session_code : 0.4597263122190395
Без  authors_code : 0.44021571536123805
Без  n_authors : 0.4579277710625568
Без  n_tags : 0.45070686856238906
Без  day : 0.45080913328725103
Без  mounth : 0.43485375897356304
Без  views_

In [226]:
0.4 * 0.6692638219008774 + 0.3 * 0.8099143688505638 + 0.3 * 0.4699530691376152

0.6516657601568047

Создаем предсказание

In [302]:
%%time
# Для view:
# session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
#        'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
#        'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#        'depth_mean_authors', 'frp_mean_authors
model_views.fit(X[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
                  'depth_mean_authors', 'frp_mean_authors']] , y['views'])
# Для depth:
#['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#        'day', 'mounth', 'views_median_cat', 'depth_median_cat',
#        'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
#        'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
#        'frp_mean_authors']
model_depth.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
                   'frp_mean_authors']], y['depth'])
# Для full_reads_percent:
# ['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#       'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
#       'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
#       'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#       'frp_mean_authors']
model_full_reads_percent.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
                                'frp_mean_authors']], y['full_reads_percent'])

Wall time: 2.5 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [54]:
test_df = df_test.drop(["title", "publish_date", "session", "tags", "authors", "category"], axis = 1)

In [55]:
test_df

,ctr,category_code,session_code,authors_code,n_authors,n_tags,day,mounth,year
document_id,,,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,0.000,0,2510,739,0,5,1,2,22
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,1.598,0,3427,739,0,4,24,5,22
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,2.006,0,4312,739,0,3,12,5,22
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,0.000,0,7405,739,0,4,19,5,22
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,9.136,0,852,494,1,3,17,2,22
...,...,...,...,...,...,...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,1.788,4,7695,358,1,4,23,3,22
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,4.427,3,7300,417,1,2,8,4,22
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,0.000,1,7214,739,0,3,14,4,22


In [303]:
sample['views'] = model_views.predict(df_test[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
                  'depth_mean_authors', 'frp_mean_authors']])
sample['depth'] = model_depth.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
                   'frp_mean_authors']])
sample['full_reads_percent'] = model_full_reads_percent.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
                                'frp_mean_authors']])

In [304]:
sample.head()

,views,depth,full_reads_percent
document_id,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,19289.992188,1.176855,39.124722
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,7442.393066,1.066307,30.939411
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,6673.118164,1.066376,35.087933
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,9913.979492,1.063834,27.311163
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,35871.843750,1.137975,22.247820


In [305]:
sample.to_csv('content/predictions/test_prediction_06_07.csv')

### Итоговый скор: 0.380437

Параша какая-то. Мб стоит убрать фигню с авторами??? Раз их так много уникальных

Пробуем с тегами, но без авторов

In [307]:
%%time
# Для view:
# session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
#        'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
#        'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#        'depth_mean_authors', 'frp_mean_authors
model_views.fit(X[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags']] , y['views'])
# Для depth:
#['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#        'day', 'mounth', 'views_median_cat', 'depth_median_cat',
#        'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
#        'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
#        'frp_mean_authors']
model_depth.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags']], y['depth'])
# Для full_reads_percent:
# ['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#       'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
#       'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
#       'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#       'frp_mean_authors']
model_full_reads_percent.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags']], y['full_reads_percent'])

Wall time: 2.94 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [309]:
sample['views'] = model_views.predict(df_test[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags']])
sample['depth'] = model_depth.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags']])
sample['full_reads_percent'] = model_full_reads_percent.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags']])

In [311]:
sample.to_csv('content/predictions/test_prediction_06_07_v2.csv')

In [317]:
sample

,views,depth,full_reads_percent
document_id,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,16038.581055,1.166056,40.904610
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,11869.832031,1.059068,30.898687
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,3040.986084,1.065905,37.054337
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,12060.156250,1.064861,36.127907
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,50512.566406,1.152337,30.235029
...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,47038.066406,1.133987,26.808678
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,25600.767578,1.090106,33.744843
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,-17806.664062,1.062526,39.309738


### Итоговый скор: 0.575

In [307]:
%%time
# Для view:
# session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
#        'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
#        'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#        'depth_mean_authors', 'frp_mean_authors
model_views.fit(X[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags']] , y['views'])
# Для depth:
#['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#        'day', 'mounth', 'views_median_cat', 'depth_median_cat',
#        'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
#        'frp_mean_tags', 'views_mean_authors', 'depth_mean_authors',
#        'frp_mean_authors']
model_depth.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags']], y['depth'])
# Для full_reads_percent:
# ['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
#       'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
#       'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
#       'depth_mean_tags', 'frp_mean_tags', 'views_mean_authors',
#       'frp_mean_authors']
model_full_reads_percent.fit(X[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags']], y['full_reads_percent'])

Wall time: 2.94 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [309]:
sample['views'] = model_views.predict(df_test[['session_code', 'authors_code', 'n_authors', 'n_tags', 'day', 'mounth',
                  'year', 'views_median_cat', 'depth_median_cat', 'views_mean_tags',
                  'depth_mean_tags', 'frp_mean_tags']])
sample['depth'] = model_depth.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                   'day', 'mounth', 'views_median_cat', 'depth_median_cat',
                   'full_reads_percent_median_cat', 'views_mean_tags', 'depth_mean_tags',
                   'frp_mean_tags']])
sample['full_reads_percent'] = model_full_reads_percent.predict(df_test[['ctr', 'category_code', 'session_code', 'authors_code', 'n_authors',
                                'n_tags', 'day', 'mounth', 'year', 'views_median_cat',
                                'depth_median_cat', 'full_reads_percent_median_cat', 'views_mean_tags',
                                'depth_mean_tags', 'frp_mean_tags']])

In [318]:
sample

,views,depth,full_reads_percent
document_id,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,16038.581055,1.166056,40.904610
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,11869.832031,1.059068,30.898687
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,3040.986084,1.065905,37.054337
628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,12060.156250,1.064861,36.127907
620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,50512.566406,1.152337,30.235029
...,...,...,...
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,47038.066406,1.133987,26.808678
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,25600.767578,1.090106,33.744843
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,-17806.664062,1.062526,39.309738


In [ ]:
sample.to_csv('content/predictions/test_prediction_07_07_v2.csv')